# Data format example

In [1]:
import pypesto
import amici
import petab

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Manage PETAB model

In [2]:
folder_base = "/home/yannik/benchmark-models/hackathon_contributions_new_data_format/"
model_name = "Zheng_PNAS2012"

manager = petab.Manager.from_folder(folder_base + model_name)

# print(petab.lint.check_measurement_df(manager.measurement_df))
manager.get_optimization_to_simulation_parameter_mapping()


[['inflowp',
  'k00_01',
  'k00_10',
  'k01_00',
  'k01_02',
  'k01_11',
  'k02_01',
  'k02_03',
  'k02_12',
  'k03_02',
  'k03_13',
  'k10_00',
  'k10_11',
  'k10_20',
  'k11_01',
  'k11_10',
  'k11_12',
  'k11_21',
  'k12_02',
  'k12_11',
  'k12_13',
  'k12_22',
  'k13_03',
  'k13_12',
  'k13_23',
  'k20_10',
  'k20_21',
  'k20_30',
  'k21_11',
  'k21_20',
  'k21_22',
  'k21_31',
  'k22_12',
  'k22_21',
  'k22_23',
  'k22_32',
  'k23_13',
  'k23_22',
  'k30_20',
  'k30_31',
  'k31_21',
  'k31_30',
  'k31_32',
  'k32_22',
  'k32_31',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma',
  'sigma']]

## Import model to AMICI

In [3]:
importer = pypesto.objective.Importer(manager)

model = importer.model
print(model.getParameterScale())
print("Model parameters:", list(model.getParameterIds()), '\n')
print("Model const parameters:", list(model.getFixedParameterIds()), '\n')
print("Model outputs:   ", list(model.getObservableIds()), '\n')
print("Model states:    ", list(model.getStateIds()), '\n')

<amici.amici.ParameterScalingVector; proxy of <Swig Object of type 'std::vector< enum amici::ParameterScaling,std::allocator< enum amici::ParameterScaling > > *' at 0x7fc267b9c600> >
Model parameters: ['inflowp', 'k00_01', 'k00_10', 'k01_00', 'k01_02', 'k01_11', 'k02_01', 'k02_03', 'k02_12', 'k03_02', 'k03_13', 'k10_00', 'k10_11', 'k10_20', 'k11_01', 'k11_10', 'k11_12', 'k11_21', 'k12_02', 'k12_11', 'k12_13', 'k12_22', 'k13_03', 'k13_12', 'k13_23', 'k20_10', 'k20_21', 'k20_30', 'k21_11', 'k21_20', 'k21_22', 'k21_31', 'k22_12', 'k22_21', 'k22_23', 'k22_32', 'k23_13', 'k23_22', 'k30_20', 'k30_31', 'k31_21', 'k31_30', 'k31_32', 'k32_22', 'k32_31', 'noiseParameter1_K27me0K36me0', 'noiseParameter1_K27me0K36me1', 'noiseParameter1_K27me0K36me2', 'noiseParameter1_K27me0K36me3', 'noiseParameter1_K27me1K36me0', 'noiseParameter1_K27me1K36me1', 'noiseParameter1_K27me1K36me2', 'noiseParameter1_K27me1K36me3', 'noiseParameter1_K27me2K36me0', 'noiseParameter1_K27me2K36me1', 'noiseParameter1_K27me2K36m

## Create objective function

In [4]:
obj, edatas = importer.create_objective()

#print(amici.getDataObservablesAsDataFrame(obj.amici_model, edatas))
#print(edatas[0].fixedParametersPreequilibration)
#print(obj.dim, obj.x_names, len(obj.x_ids), obj.opt_to_sim_par_mapping)
#print(edatas[0].fixedParametersPreequilibration, edatas[0].fixedParameters)

print("Nominal parameter values:\n", manager.x_nominal)

obj(manager.x_nominal)

PARAMETER MAPPING: [['inflowp', 'k00_01', 'k00_10', 'k01_00', 'k01_02', 'k01_11', 'k02_01', 'k02_03', 'k02_12', 'k03_02', 'k03_13', 'k10_00', 'k10_11', 'k10_20', 'k11_01', 'k11_10', 'k11_12', 'k11_21', 'k12_02', 'k12_11', 'k12_13', 'k12_22', 'k13_03', 'k13_12', 'k13_23', 'k20_10', 'k20_21', 'k20_30', 'k21_11', 'k21_20', 'k21_22', 'k21_31', 'k22_12', 'k22_21', 'k22_23', 'k22_32', 'k23_13', 'k23_22', 'k30_20', 'k30_31', 'k31_21', 'k31_30', 'k31_32', 'k32_22', 'k32_31', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma']]
SCALE MAPPING: [['log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10

-278.3335326998418

## Run optimization

In [5]:
optimizer = pypesto.ScipyOptimizer()

problem = importer.create_problem(obj)

# do the optimization
result = pypesto.minimize(problem=problem, optimizer=optimizer,
                          n_starts=10)

TypeError: can't pickle SwigPyObject objects

## Visualize

In [ ]:
import pypesto.visualize

pypesto.visualize.waterfall(result)
pypesto.visualize.parameters(result)

print(result.optimize_result.get_for_key('fval'))

In [ ]:
df = obj.simulations_to_measurement_df(result.optimize_result.get_for_key('x')[0])
plt.xlabel("Experiment")
plt.ylabel("Simulation")
plt.scatter(importer.measurement_df['measurement'], df['measurement'])

In [ ]:
edatas_sim = obj.simulate(x_bst)
df_sim = importer.to_measurement_df(edatas_sim)
plot_simulation(manager.measurement_df, [df_sim])